# Spotify API Request and Data Cleaning
By Alec Plante <br><br>
This notebook shows the process of how I got data from the spotify API and cleaned it. <br> 
NOTE: this code is relatively complex when compared to the other code in the project

## Imports

In [1]:
# Log in credientials for my spotify API account
import AccountInfo

In [2]:
# import spotippy (the spotify API Python Library)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Validate my credentials with sp 
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= AccountInfo.client_id,
                                                           client_secret= AccountInfo.client_secret))

In [3]:
# Import libraries for data analysis
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np

In [181]:
# import data for code randomization
import random
import math

## Load data and clean data
... from Kworb (wikipedias source)

In [4]:
# Load the data I scraped from Kworb
topsongs = pd.read_csv('Data/spotifyTopSongs.tsv')

In [5]:
#view the data to make sure it imported well
topsongs.head()

,Unnamed: 0,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total
0,0,1,The Weeknd,Blinding Lights,172,64.0,1,(x13),52375259,3406980588
1,1,2,Ed Sheeran,Shape of You,308,34.0,1,(x14),64217796,3302350300
2,2,3,Lewis Capaldi,Someone You Loved,218,15.0,4,NaN,24962682,2623186016
3,3,4,Tones And I,Dance Monkey,151,41.0,1,(x17),52055226,2538070349
4,4,5,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,225,31.0,1,(x2),34579416,2498983472


In [6]:
# Drop the first column, because its a redundent index column
topsongs.drop("Unnamed: 0", axis = 1, inplace=True)

In [29]:
# View the data again to make sure it looks good
topsongs.head()

,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total,info,items,artists
0,1,The Weeknd,Blinding Lights,172,64.0,1,(x13),52375259,3406980588,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'1Xyo4u8uXC1ZmMpatF05PJ': 'The Weeknd'}]
1,2,Ed Sheeran,Shape of You,308,34.0,1,(x14),64217796,3302350300,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'6eUKZXaKkcviH0Ku9w2n3V': 'Ed Sheeran'}]
2,3,Lewis Capaldi,Someone You Loved,218,15.0,4,NaN,24962682,2623186016,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'4GNC7GD6oZMSxPGyXy4MNB': 'Lewis Capaldi'}]
3,4,Tones And I,Dance Monkey,151,41.0,1,(x17),52055226,2538070349,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'single', 'album_typ...",[{'2NjfBq1NflQcKSeiDooVjY': 'Tones And I'}]
4,5,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,225,31.0,1,(x2),34579416,2498983472,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'compilation', 'albu...","[{'246dkjvS1zLTtiykXe5h60': 'Post Malone'}, {'..."


## Spotify API Data Pull and Cleaning

I will be pulling data from the Spotify API based on these top streamed songs from Kworb.

Here are the steps to get the data on songs that I want:
1. Request the data from the Spotify API and save the results to a column for every song based on the song title and artist
2. Save the basic data as a column for each track, including artists, explicit status, etc.
3. Request the advanaced features for each song (dancablility, loudness, instrumentalenss, etc.)
4. Save each Feature to its own column


NOTE: Spotify does NOT let you pull **total stream numbers** with its API. To obtain streaming data, one can webscrape it. In the interest of time, I will not be scraping this data. However, you can use **POPULARITY:** which takes into account total number of streams and the recency of those streams. Because we are discovering how to make a hit NOW (and not 5 years ago), this metric may be more beneficial than total number of streams.

The API also does not associate a song with a particular genre. That information is gotten from the artists information.



#### Step 1: Request the data from the Spotify API and save the results to a column

In [8]:
# New column 'info' that is the information from that song returned from the spotify API

# This code says: for each song/aritst combo, get the API information and add it as its own column
topsongs['info'] = topsongs.apply(lambda x: sp.search(q=f"track: \
                \"{x['Title']}\" artist: \"{x['Artist']}\"", limit = 1), axis = 1)
# The result of this search will be JSON like structures of each row.
# we can get data about the song from this

In [202]:
#print what this data will look like
topsongs['info'][0]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A+%22Blinding+Lights%22+artist%3A+%22The+Weeknd%22&type=track&offset=0&limit=1',
  'items': [{'album': {'album_group': 'album',
     'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
   

In [25]:
# Create new column items that grabs the 'items' portion of the info structure.
# for each row, get the info portiion if it exists, otherwise mark as NaN
topsongs['items'] = topsongs.apply(lambda x: x['info']['tracks']['items'][0] if len(x['info']['tracks']['items'])==1 else np.nan, axis = 1)

#This makes the JSON easier to deal with

The search for track data may have come up empty. To maintain inegrety of data, we need to check this.

In [27]:
# to see if there are any times spotify did not return a result
topsongs[topsongs['items'].isna()]

,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total,info,items,artists
5283,5284,Samir & Viktor,Groupie,4,NaN,146,NaN,1469139,4955784,{'tracks': {'href': 'https://api.spotify.com/v...,NaN,NaN


The only song in this list that the query could not find is Groupie by Samir & Viktor. Upon further investigation, this song is undiscoverable because it is unvailable in my current region. For now, I will mark it as NaN but will consider removing.

#### Step 2: Save the basic data as a column for each track, including artists, explicit status, etc.
Now we have this 'items' structure that we can get basic information about the song. They are nested in this structure

In [28]:
#artists column that lists artist names and ID in dictionary format
topsongs['artists'] = topsongs.apply(lambda x: [{i['id']:i['name']}for i in x['items']['artists']] if type(x['items']) == dict else np.nan, axis = 1)

In [30]:
#add a songids column to the table
topsongs['songid'] = topsongs.apply(lambda x: x['items']['id'] \
                            if type(x['items']) == dict else np.nan, axis = 1)

In [33]:
#add a popularity column to the table
topsongs['popularity'] = topsongs.apply(lambda x: x['items']['popularity'] \
                                if  type(x['items']) == dict else np.nan, axis = 1)

In [42]:
#add whether or not the track is explicit to the table
topsongs['explicit'] = topsongs.apply(lambda x: x['items']['explicit'] \
                            if type(x['items']) == dict else np.nan, axis = 1)

In [176]:
# add a release date to the table
topsongs['release_date'] = topsongs.apply(lambda x: x['items']['album']['release_date'] \
                                    if type(x['items']) == dict else np.nan, axis = 1)

The basic information has now all been added as a column. Now we'll move on to the more advanced features

#### Step 3: Request the advanaced features for each song (dancablility, loudness, instrumentalenss, etc.)
The first section will be testing the result, and the second will be implementing the feature dictionary as a column

In [45]:
# Grab the data for Blining lights by The Weeknd
test = sp.search(q=f"track: \"{'Blinding Lights'}\" artist: \"{'The Weeknd'}\"", limit = 1)


In [48]:
# Get the ID for the song Blinding Lights
testid=test['tracks']['items'][0]['id']

In [49]:
# Get the features for the tracks based on the id
test_features = sp.audio_features(testid)

In [50]:
#view the features for Blinding Lights By the Weeknd
test_features

[{'danceability': 0.514,
  'energy': 0.73,
  'key': 1,
  'loudness': -5.934,
  'mode': 1,
  'speechiness': 0.0598,
  'acousticness': 0.00146,
  'instrumentalness': 9.54e-05,
  'liveness': 0.0897,
  'valence': 0.334,
  'tempo': 171.005,
  'type': 'audio_features',
  'id': '0VjIjW4GlUZAMYd2vXMi3b',
  'uri': 'spotify:track:0VjIjW4GlUZAMYd2vXMi3b',
  'track_href': 'https://api.spotify.com/v1/tracks/0VjIjW4GlUZAMYd2vXMi3b',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0VjIjW4GlUZAMYd2vXMi3b',
  'duration_ms': 200040,
  'time_signature': 4}]

In [51]:
#list the features I want to use for data analysis
featList = ['danceability','energy','key','loudness','mode','speechiness',\
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature']
for feat in featList:
    print(test_features[0][feat])


0.514
0.73
1
-5.934
1
0.0598
0.00146
9.54e-05
0.0897
0.334
171.005
200040
4


Now, I can find the features for all of the songs in the table based on the song ID

In [54]:
# Features column for a dictionary of features for each song
topsongs['features'] = topsongs.apply(lambda x: sp.audio_features(x['songid'])[0] \
                                      if type(x['items']) == dict else np.nan, axis = 1)

In [77]:
# See if any rows did not gather any data for features
topsongs[topsongs['features'].apply(lambda x: not(isinstance(x, dict)))]

,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total,info,items,artists,songid,popularity,explicit,features
3705,3706,Pop Smoke,Bout A Million,2,NaN,65,NaN,8961078,14276812,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'compilation', 'albu...","[{'0eDvMgVFoNV3TpwtrVCoTj': 'Pop Smoke'}, {'1U...",6foz9Xzoi7YvPBTWSkf7wx,0.0,True,None
5283,5284,Samir & Viktor,Groupie,4,NaN,146,NaN,1469139,4955784,{'tracks': {'href': 'https://api.spotify.com/v...,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
#this song has no available audio features. Add catch and set to NaN
sp.audio_features('6foz9Xzoi7YvPBTWSkf7wx')

[None]

In [100]:
#change the None to nan
topsongs.loc[pd.isnull(topsongs['features']), 'features'] = np.nan

In [101]:
#see results of change
topsongs[pd.isnull(topsongs['features'])]

,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total,info,items,artists,songid,popularity,explicit,features
3705,3706,Pop Smoke,Bout A Million,2,NaN,65,NaN,8961078,14276812,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'compilation', 'albu...","[{'0eDvMgVFoNV3TpwtrVCoTj': 'Pop Smoke'}, {'1U...",6foz9Xzoi7YvPBTWSkf7wx,0.0,True,NaN
5283,5284,Samir & Viktor,Groupie,4,NaN,146,NaN,1469139,4955784,{'tracks': {'href': 'https://api.spotify.com/v...,NaN,NaN,NaN,NaN,NaN,NaN


#### Step 4: Save each Feature to its own column

In [104]:
# Create a list of features to be used in analysis, and make a column for each feature:
featList = ['danceability','energy','key','loudness','mode','speechiness', \
            'acousticness', 'instrumentalness', 'liveness', 'valence', \
            'tempo','duration_ms', 'time_signature']

# For Feature in list, make column that value of that feature for every song
for feat in featList:
    topsongs[feat] = topsongs.apply(lambda x: x['features'][feat] if type(x['features']) == dict else np.nan, axis = 1)

In [177]:
# View changes
topsongs.head()

,Pos,Artist,Title,Wks,T10,Pk,(x?),PkStreams,Total,info,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_date
0,1,The Weeknd,Blinding Lights,172,64.0,1,(x13),52375259,3406980588,{'tracks': {'href': 'https://api.spotify.com/v...,...,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,200040.0,4.0,2020-03-20
1,2,Ed Sheeran,Shape of You,308,34.0,1,(x14),64217796,3302350300,{'tracks': {'href': 'https://api.spotify.com/v...,...,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,233713.0,4.0,2017-03-03
2,3,Lewis Capaldi,Someone You Loved,218,15.0,4,NaN,24962682,2623186016,{'tracks': {'href': 'https://api.spotify.com/v...,...,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,182161.0,4.0,2019-05-17
3,4,Tones And I,Dance Monkey,151,41.0,1,(x17),52055226,2538070349,{'tracks': {'href': 'https://api.spotify.com/v...,...,0.0,0.0924,0.69200,0.000104,0.1490,0.513,98.027,209438.0,4.0,2019-10-17
4,5,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,225,31.0,1,(x2),34579416,2498983472,{'tracks': {'href': 'https://api.spotify.com/v...,...,1.0,0.0466,0.55600,0.000000,0.0703,0.913,89.911,158040.0,4.0,2018-12-14


In [106]:
#Check to see if I have all the wanted columns
topsongs.columns
#Looks good

Index(['Pos', 'Artist', 'Title', 'Wks', 'T10', 'Pk', '(x?)', 'PkStreams',
       'Total', 'info', 'items', 'artists', 'songid', 'popularity', 'explicit',
       'features', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [178]:
#save to a TSV file so I dont have to deal with the JSON type files anymore
topsongs.to_csv('Data/spotifyTopSongsWithAPI.tsv')

## Random data pulling


The topsongs column now has all of the data wanted for the top songs that we pulled from Kworb.

In order to get a balanced dataset, we'll attempt to grab a **random sample** of songs from the Spotify API. There is not native random sample function for the spotify API. I will try to randomize the search query to obtain a random sample.

When getting this new data from the API, some of the columns in topsongs will not be available, like pos, wks, T10, etc. To easily integrate that data, I will create a stripped down version of the topsongs table.

In [204]:
colOrder = ['Artist', 'Title', 'info', 'items', 'artists', 'songid', 'popularity', 'explicit', 'release_date', 'features', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms', 'time_signature']

In [205]:
topsongsstripped=topsongs.loc[:,colOrder]

In [183]:
randData = pd.DataFrame(columns = topsongsstripped.columns)

In [184]:
randData.columns

Index(['Artist', 'Title', 'info', 'items', 'artists', 'songid', 'popularity',
       'explicit', 'release_date', 'features', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'],
      dtype='object')

In [203]:
topsongs.loc[:,colOrder]

,Artist,Title,info,items,artists,songid,popularity,explicit,release_date,features,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,The Weeknd,Blinding Lights,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'1Xyo4u8uXC1ZmMpatF05PJ': 'The Weeknd'}],0VjIjW4GlUZAMYd2vXMi3b,90.0,False,2020-03-20,"{'danceability': 0.514, 'energy': 0.73, 'key':...",...,-5.934,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,200040.0,4.0
1,Ed Sheeran,Shape of You,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'6eUKZXaKkcviH0Ku9w2n3V': 'Ed Sheeran'}],7qiZfU4dY1lWllzX7mPBI3,86.0,False,2017-03-03,"{'danceability': 0.825, 'energy': 0.652, 'key'...",...,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,233713.0,4.0
2,Lewis Capaldi,Someone You Loved,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'4GNC7GD6oZMSxPGyXy4MNB': 'Lewis Capaldi'}],7qEHsqek33rTcFNT9PFqLf,87.0,False,2019-05-17,"{'danceability': 0.501, 'energy': 0.405, 'key'...",...,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,182161.0,4.0
3,Tones And I,Dance Monkey,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'single', 'album_typ...",[{'2NjfBq1NflQcKSeiDooVjY': 'Tones And I'}],2XU0oxnq2qxCpomAAuJY8K,77.0,False,2019-10-17,"{'danceability': 0.824, 'energy': 0.588, 'key'...",...,-6.400,0.0,0.0924,0.69200,0.000104,0.1490,0.513,98.027,209438.0,4.0
4,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'compilation', 'albu...","[{'246dkjvS1zLTtiykXe5h60': 'Post Malone'}, {'...",3KkXRkHbMCARz0aVfEt68P,81.0,False,2018-12-14,"{'danceability': 0.76, 'energy': 0.479, 'key':...",...,-5.574,1.0,0.0466,0.55600,0.000000,0.0703,0.913,89.911,158040.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6175,Mariah Carey,Infinity,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'compilation', 'albu...",[{'4iHNK0tOyZPYnBU7nGAgpQ': 'Mariah Carey'}],3qczyRtur0tILibW9rHBFA,30.0,False,2015-05-15,"{'danceability': 0.55, 'energy': 0.805, 'key':...",...,-4.327,1.0,0.0862,0.05200,0.000003,0.2810,0.217,134.755,239800.0,4.0
6176,Twenty One Pilots,Fairly Local,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'3YQKmKGau1PzlVlkL1iodx': 'Twenty One Pilots'}],1s9xuJxhYM0LdVqksjegcg,61.0,False,2015-05-15,"{'danceability': 0.715, 'energy': 0.723, 'key'...",...,-5.993,0.0,0.0569,0.03960,0.000086,0.0840,0.479,114.985,207160.0,4.0
6177,Ludacris,Good Lovin,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...","[{'3ipn9JLAPI5GUEo4y4jcoi': 'Ludacris'}, {'360...",3aeSTu6ejN2BU8YywEcHWO,53.0,True,2015-03-31,"{'danceability': 0.619, 'energy': 0.754, 'key'...",...,-5.231,1.0,0.2320,0.29400,0.000003,0.1090,0.589,83.464,223440.0,4.0
6178,iSHi,Push It,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...","[{'6Dy7Ex5zlTLgXbsB92rPji': 'iSHi'}, {'2rRz9oF...",3KIzaWqMM6wjgVVfcfU5Uj,34.0,True,2021-12-03,"{'danceability': 0.628, 'energy': 0.742, 'key'...",...,-6.231,0.0,0.1400,0.01220,0.247000,0.1220,0.184,175.042,230187.0,4.0


The new table is now created. The next step is to populate it with random spotify data. The cell below accomplishes that. 

The cell below is complicated, but at a high level, it grabs a pseudo random spotify song using random letters and an offset value (which returns the nth item from the search queue)

In [207]:
## Get random tracks

## number of random songs you want to grab
numberOfRandomSongs = 5000
## Characters to pick a random one and search for
characters = 'abcdefghijklmnopqrstuvwxyz'

## For 5000 songs, repeat the process below
#### Generate a random search query
#### Look that query up for the spotify API
#### Create a list and append all of the wanted features to it
#### Add the row to the RandData dataframe

for i in range(numberOfRandomSongs):
    
    ######## Random Search Query #########
    # Get random character to look up
    randChar = characters[math.floor(random.random()*len(characters))]
    # Add wildcard for search param (random if first letter of search or not)
    randChar = round(random.random())*'%' +randChar + '%' 
    # get random offset (nth result in search queue)
    randOffset = math.floor(random.random()*1000)
    print(f"INIT {i}: randChar: {randChar}, randOffset: {randOffset}")
    # Search using parameters genreated above
    randInfo = sp.search(q=f"track:{randChar}", limit = 1, offset=randOffset)
    
    # If there is no data, restart loop
    if(randInfo['tracks']['items'][0] == None):
        continue
        
        
        
    ###### EXTRACT DATA FROM INFO ########
    #title
    randTitle = randInfo['tracks']['items'][0]['name']
    #Artist
    randArtist = randInfo['tracks']['items'][0]['artists'][0]['name']
    #Start row to add in with these variables. Append other later
    randRow = [randArtist,randTitle, randInfo]
    #random 'items' part of JSON like strucutre 
    randItems = randInfo['tracks']['items'][0]
    randRow.append(randItems)
    # artists and IDs
    randRow.append(list([{i['id']:i['name']} for i in randItems['artists']]))
    # Song id
    randRow.append(randItems['id'])
    # popularity
    randRow.append(randItems['popularity'])
    # explicit rating
    randRow.append(randItems['explicit'])
    # release date
    randRow.append(randItems['album']['release_date'])
    
    # Get list of features from spotify API 
    randFeat = sp.audio_features(randItems['id'])[0]
    
    # if no results, fill features columns with NaN
    if randFeat == None:
        randRow.append(np.nan)
        for feature in range(len(featList)):
            randRow.append(np.nan)
    else: #else, fill with corresponding feature values
        randRow.append(randFeat)
        for feature in featList:
            randRow.append(randFeat[feature])
    # add the row to dataframe
    randData.loc[len(randData.index)] = randRow
    


INIT 0: randChar: i%, randOffset: 795
INIT 1: randChar: %u%, randOffset: 775
INIT 2: randChar: t%, randOffset: 878
INIT 3: randChar: b%, randOffset: 440
INIT 4: randChar: %v%, randOffset: 442
INIT 5: randChar: w%, randOffset: 180
INIT 6: randChar: l%, randOffset: 518
INIT 7: randChar: %y%, randOffset: 470
INIT 8: randChar: g%, randOffset: 830
INIT 9: randChar: i%, randOffset: 64
INIT 10: randChar: %q%, randOffset: 178
INIT 11: randChar: e%, randOffset: 364
INIT 12: randChar: t%, randOffset: 287
INIT 13: randChar: %x%, randOffset: 216
INIT 14: randChar: k%, randOffset: 880
INIT 15: randChar: %j%, randOffset: 986
INIT 16: randChar: i%, randOffset: 942
INIT 17: randChar: d%, randOffset: 364
INIT 18: randChar: l%, randOffset: 662
INIT 19: randChar: %b%, randOffset: 33
INIT 20: randChar: %c%, randOffset: 301
INIT 21: randChar: y%, randOffset: 876
INIT 22: randChar: k%, randOffset: 454
INIT 23: randChar: e%, randOffset: 983
INIT 24: randChar: %h%, randOffset: 965
INIT 25: randChar: k%, randO

INIT 206: randChar: m%, randOffset: 537
INIT 207: randChar: %r%, randOffset: 545
INIT 208: randChar: %c%, randOffset: 887
INIT 209: randChar: r%, randOffset: 13
INIT 210: randChar: n%, randOffset: 468
INIT 211: randChar: k%, randOffset: 476
INIT 212: randChar: %l%, randOffset: 954
INIT 213: randChar: n%, randOffset: 328
INIT 214: randChar: %y%, randOffset: 334
INIT 215: randChar: a%, randOffset: 458
INIT 216: randChar: t%, randOffset: 323
INIT 217: randChar: %q%, randOffset: 967
INIT 218: randChar: %m%, randOffset: 363
INIT 219: randChar: v%, randOffset: 594
INIT 220: randChar: %j%, randOffset: 187
INIT 221: randChar: %k%, randOffset: 652
INIT 222: randChar: e%, randOffset: 879
INIT 223: randChar: %q%, randOffset: 503
INIT 224: randChar: %u%, randOffset: 599
INIT 225: randChar: e%, randOffset: 729
INIT 226: randChar: a%, randOffset: 797
INIT 227: randChar: q%, randOffset: 779
INIT 228: randChar: %r%, randOffset: 307
INIT 229: randChar: w%, randOffset: 769
INIT 230: randChar: %x%, randO

INIT 409: randChar: h%, randOffset: 220
INIT 410: randChar: %f%, randOffset: 999
INIT 411: randChar: x%, randOffset: 667
INIT 412: randChar: %v%, randOffset: 577
INIT 413: randChar: j%, randOffset: 992
INIT 414: randChar: %m%, randOffset: 563
INIT 415: randChar: %h%, randOffset: 544
INIT 416: randChar: m%, randOffset: 161
INIT 417: randChar: l%, randOffset: 696
INIT 418: randChar: %j%, randOffset: 924
INIT 419: randChar: e%, randOffset: 503
INIT 420: randChar: d%, randOffset: 315
INIT 421: randChar: e%, randOffset: 69
INIT 422: randChar: %v%, randOffset: 747
INIT 423: randChar: %e%, randOffset: 459
INIT 424: randChar: %d%, randOffset: 665
INIT 425: randChar: r%, randOffset: 345
INIT 426: randChar: g%, randOffset: 134
INIT 427: randChar: %g%, randOffset: 215
INIT 428: randChar: f%, randOffset: 464
INIT 429: randChar: %b%, randOffset: 807
INIT 430: randChar: v%, randOffset: 456
INIT 431: randChar: %m%, randOffset: 666
INIT 432: randChar: m%, randOffset: 939
INIT 433: randChar: x%, randOf

INIT 612: randChar: s%, randOffset: 421
INIT 613: randChar: %l%, randOffset: 453
INIT 614: randChar: %a%, randOffset: 777
INIT 615: randChar: %r%, randOffset: 966
INIT 616: randChar: u%, randOffset: 57
INIT 617: randChar: %r%, randOffset: 37
INIT 618: randChar: h%, randOffset: 197
INIT 619: randChar: %a%, randOffset: 242
INIT 620: randChar: c%, randOffset: 772
INIT 621: randChar: %w%, randOffset: 121
INIT 622: randChar: f%, randOffset: 805
INIT 623: randChar: u%, randOffset: 392
INIT 624: randChar: c%, randOffset: 771
INIT 625: randChar: r%, randOffset: 606
INIT 626: randChar: %j%, randOffset: 442
INIT 627: randChar: z%, randOffset: 824
INIT 628: randChar: s%, randOffset: 944
INIT 629: randChar: %j%, randOffset: 808
INIT 630: randChar: z%, randOffset: 715
INIT 631: randChar: m%, randOffset: 527
INIT 632: randChar: b%, randOffset: 47
INIT 633: randChar: %t%, randOffset: 799
INIT 634: randChar: %l%, randOffset: 572
INIT 635: randChar: %m%, randOffset: 1
INIT 636: randChar: %r%, randOffse

INIT 816: randChar: y%, randOffset: 801
INIT 817: randChar: %n%, randOffset: 823
INIT 818: randChar: %b%, randOffset: 445
INIT 819: randChar: %q%, randOffset: 440
INIT 820: randChar: %v%, randOffset: 413
INIT 821: randChar: %r%, randOffset: 330
INIT 822: randChar: %j%, randOffset: 949
INIT 823: randChar: h%, randOffset: 517
INIT 824: randChar: n%, randOffset: 685
INIT 825: randChar: %w%, randOffset: 319
INIT 826: randChar: f%, randOffset: 186
INIT 827: randChar: %s%, randOffset: 125
INIT 828: randChar: %y%, randOffset: 600
INIT 829: randChar: %p%, randOffset: 850
INIT 830: randChar: l%, randOffset: 849
INIT 831: randChar: %l%, randOffset: 268
INIT 832: randChar: r%, randOffset: 502
INIT 833: randChar: %o%, randOffset: 784
INIT 834: randChar: %c%, randOffset: 977
INIT 835: randChar: %x%, randOffset: 524
INIT 836: randChar: m%, randOffset: 801
INIT 837: randChar: %c%, randOffset: 418
INIT 838: randChar: a%, randOffset: 561
INIT 839: randChar: %l%, randOffset: 894
INIT 840: randChar: c%, 

...

INIT 4193: randChar: a%, randOffset: 211
INIT 4194: randChar: x%, randOffset: 380
INIT 4195: randChar: %q%, randOffset: 361
INIT 4196: randChar: y%, randOffset: 939
INIT 4197: randChar: x%, randOffset: 994
INIT 4198: randChar: %a%, randOffset: 232
INIT 4199: randChar: a%, randOffset: 40
INIT 4200: randChar: %y%, randOffset: 546
INIT 4201: randChar: %a%, randOffset: 620
INIT 4202: randChar: l%, randOffset: 319
INIT 4203: randChar: y%, randOffset: 557
INIT 4204: randChar: n%, randOffset: 255
INIT 4205: randChar: u%, randOffset: 891
INIT 4206: randChar: o%, randOffset: 35
INIT 4207: randChar: %l%, randOffset: 593
INIT 4208: randChar: %l%, randOffset: 963
INIT 4209: randChar: p%, randOffset: 323
INIT 4210: randChar: %l%, randOffset: 123
INIT 4211: randChar: %a%, randOffset: 724
INIT 4212: randChar: o%, randOffset: 705
INIT 4213: randChar: p%, randOffset: 73
INIT 4214: randChar: b%, randOffset: 571
INIT 4215: randChar: %y%, randOffset: 801
INIT 4216: randChar: f%, randOffset: 677
INIT 4217:

INIT 4392: randChar: %j%, randOffset: 563
INIT 4393: randChar: l%, randOffset: 965
INIT 4394: randChar: v%, randOffset: 958
INIT 4395: randChar: %r%, randOffset: 316
INIT 4396: randChar: b%, randOffset: 669
INIT 4397: randChar: u%, randOffset: 443
INIT 4398: randChar: n%, randOffset: 582
INIT 4399: randChar: %y%, randOffset: 7
INIT 4400: randChar: b%, randOffset: 237
INIT 4401: randChar: x%, randOffset: 446
INIT 4402: randChar: h%, randOffset: 835
INIT 4403: randChar: t%, randOffset: 756
INIT 4404: randChar: %u%, randOffset: 920
INIT 4405: randChar: z%, randOffset: 271
INIT 4406: randChar: n%, randOffset: 554
INIT 4407: randChar: n%, randOffset: 189
INIT 4408: randChar: r%, randOffset: 600
INIT 4409: randChar: d%, randOffset: 495
INIT 4410: randChar: e%, randOffset: 899
INIT 4411: randChar: %b%, randOffset: 111
INIT 4412: randChar: k%, randOffset: 281
INIT 4413: randChar: y%, randOffset: 458
INIT 4414: randChar: %q%, randOffset: 600
INIT 4415: randChar: %w%, randOffset: 501
INIT 4416: 

INIT 4590: randChar: %c%, randOffset: 284
INIT 4591: randChar: %x%, randOffset: 907
INIT 4592: randChar: %p%, randOffset: 408
INIT 4593: randChar: %n%, randOffset: 812
INIT 4594: randChar: %x%, randOffset: 831
INIT 4595: randChar: %c%, randOffset: 925
INIT 4596: randChar: %d%, randOffset: 326
INIT 4597: randChar: g%, randOffset: 762
INIT 4598: randChar: %p%, randOffset: 317
INIT 4599: randChar: %b%, randOffset: 349
INIT 4600: randChar: f%, randOffset: 850
INIT 4601: randChar: f%, randOffset: 57
INIT 4602: randChar: %r%, randOffset: 47
INIT 4603: randChar: e%, randOffset: 247
INIT 4604: randChar: s%, randOffset: 62
INIT 4605: randChar: z%, randOffset: 510
INIT 4606: randChar: x%, randOffset: 843
INIT 4607: randChar: %s%, randOffset: 614
INIT 4608: randChar: %v%, randOffset: 65
INIT 4609: randChar: a%, randOffset: 965
INIT 4610: randChar: t%, randOffset: 385
INIT 4611: randChar: k%, randOffset: 261
INIT 4612: randChar: y%, randOffset: 573
INIT 4613: randChar: q%, randOffset: 503
INIT 461

INIT 4789: randChar: %m%, randOffset: 141
INIT 4790: randChar: h%, randOffset: 373
INIT 4791: randChar: y%, randOffset: 19
INIT 4792: randChar: %e%, randOffset: 548
INIT 4793: randChar: y%, randOffset: 580
INIT 4794: randChar: d%, randOffset: 104
INIT 4795: randChar: %k%, randOffset: 776
INIT 4796: randChar: %q%, randOffset: 978
INIT 4797: randChar: %b%, randOffset: 266
INIT 4798: randChar: %t%, randOffset: 624
INIT 4799: randChar: %u%, randOffset: 184
INIT 4800: randChar: x%, randOffset: 934
INIT 4801: randChar: m%, randOffset: 383
INIT 4802: randChar: x%, randOffset: 696
INIT 4803: randChar: y%, randOffset: 938
INIT 4804: randChar: %g%, randOffset: 722
INIT 4805: randChar: y%, randOffset: 464
INIT 4806: randChar: t%, randOffset: 6
INIT 4807: randChar: c%, randOffset: 893
INIT 4808: randChar: %s%, randOffset: 409
INIT 4809: randChar: r%, randOffset: 728
INIT 4810: randChar: l%, randOffset: 335
INIT 4811: randChar: s%, randOffset: 6
INIT 4812: randChar: n%, randOffset: 374
INIT 4813: r

INIT 4988: randChar: u%, randOffset: 464
INIT 4989: randChar: %s%, randOffset: 306
INIT 4990: randChar: b%, randOffset: 807
INIT 4991: randChar: %h%, randOffset: 773
INIT 4992: randChar: %i%, randOffset: 542
INIT 4993: randChar: z%, randOffset: 38
INIT 4994: randChar: %n%, randOffset: 116
INIT 4995: randChar: t%, randOffset: 243
INIT 4996: randChar: %q%, randOffset: 368
INIT 4997: randChar: %w%, randOffset: 938
INIT 4998: randChar: %k%, randOffset: 713
INIT 4999: randChar: x%, randOffset: 384


In [196]:
#view the random data
randData

,Artist,Title,info,items,artists,songid,popularity,explicit,release_date,features,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Crystal Castles,Untrust Us,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'7K3zpFXBvPcvzhj7zlGJdO': 'Crystal Castles'}],6RpuNjTsUQo7eByN3kWvYx,62,False,2008-03-18,"{'danceability': 0.609, 'energy': 0.807, 'key'...",...,-5.405,0,0.0283,0.037600,0.204000,0.1300,0.2030,125.906,186373,4
1,Surf Mesa,ily (i love you baby) (feat. Emilee),{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'single', 'album_typ...","[{'1lmU3giNF3CSbkVSQmLpHQ': 'Surf Mesa'}, {'4A...",62aP9fBQKYKxi7PDXwcUAS,75,False,2019-11-26,"{'danceability': 0.674, 'energy': 0.774, 'key'...",...,-7.567,0,0.0892,0.068600,0.001880,0.3930,0.3300,112.050,176547,4
2,Taylor Swift,champagne problems,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'06HL4z0CvFAxyc27GXpf02': 'Taylor Swift'}],1gcyHQpBQ1lfXGdhZmWrHP,69,True,2021-01-07,"{'danceability': 0.462, 'energy': 0.24, 'key':...",...,-12.077,1,0.0377,0.920000,0.000000,0.1130,0.3200,171.319,244000,4
3,Dua Lipa,Levitating (feat. DaBaby),{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...","[{'6M2wZ9GZgrQXHCFfjv46we': 'Dua Lipa'}, {'4r6...",5nujrmhLynf4yMoMtj8AQF,83,False,2020-03-27,"{'danceability': 0.702, 'energy': 0.825, 'key'...",...,-3.787,0,0.0601,0.008830,0.000000,0.0674,0.9150,102.977,203064,4
4,Angel Y Khriz,Na De Na,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...","[{'7npqy7rgdX2D3yfVyWExKo': 'Angel Y Khriz'}, ...",2Jz1X6ZB0gkkCVgmeVGVGp,72,True,2008,"{'danceability': 0.878, 'energy': 0.894, 'key'...",...,-5.834,1,0.1230,0.184000,0.000000,0.1050,0.6820,119.927,202867,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4560,Big L,Danger Zone,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'30fiiwr2EeZp1tAhzYCmyM': 'Big L'}],0kU9oWqYnPeqEca3x2auV9,43,True,1995-03-28,"{'danceability': 0.521, 'energy': 0.481, 'key'...",...,-11.371,1,0.2950,0.000325,0.000001,0.2780,0.3410,170.980,216600,4
4561,Andra Day,Rise Up,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'1c4rxrxy8eDLvMVL1DTiBe': 'Andra Day'}],0tV8pOpiNsKqUys0ilUcXz,72,False,2015-08-28,"{'danceability': 0.398, 'energy': 0.316, 'key'...",...,-7.031,1,0.0368,0.805000,0.000000,0.2950,0.1740,118.582,253342,4
4562,Sleepy Hallow,Breakin Bad (Okay),{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...","[{'6EPlBSH2RSiettczlz7ihV': 'Sleepy Hallow'}, ...",2KMPEI406H9G8UqiWDfuTK,66,True,2019-10-31,"{'danceability': 0.748, 'energy': 0.62, 'key':...",...,-5.631,0,0.3490,0.340000,0.000000,0.1500,0.9150,94.508,131287,4
4563,Luke Combs,A Song Was Born,{'tracks': {'href': 'https://api.spotify.com/v...,"{'album': {'album_group': 'album', 'album_type...",[{'718COspgdWOnwOFpJHRZHS': 'Luke Combs'}],4AhZSJyGTfYqUnFjn5eGQa,64,False,2023-03-24,"{'danceability': 0.583, 'energy': 0.787, 'key'...",...,-4.230,1,0.0483,0.036400,0.000007,0.3130,0.6730,155.921,189947,4


In [197]:
# Export Stripped songs to tsv
topsongsstripped.to_csv('Data/topSpotifyStripped.tsv')

In [198]:
# Export Original Topsongs to tsv
topsongs.to_csv('Data/spotifyTopSongsWithAPI.tsv')

In [209]:
# Export random songs to tsv
randData.to_csv('Data/topSpotifyStrippedRand.tsv')

The Kworb based data and randome data are now both taken from the API and Stored in TSV files ready to be imported into the main project. I had more plans with this section, but due to time constraints, I was unable to complete. If you're curious, check out the raw code file for the API interactions